In [1]:
#!pip install chesslab

In [2]:
import numpy as np
from chesslab.utils import load_pkl
from chesslab.training_tf import fitting
from sklearn.model_selection import train_test_split
import tensorflow as tf

In [3]:

lr = 0.1
epochs=20
batch_size = 128
test_percent=0.1

path = 'D:/database/ccrl/'
name_data='ccrl_states_elo3.pkl'
name_labels='ccrl_results_elo3.pkl'
save_name='./tmp/tf_weights-relu-elo3.4'

optim = tf.compat.v1.train.GradientDescentOptimizer(learning_rate=lr)
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

In [4]:
download=False
if download:
    from chesslab.utils import download_7z
    path='./'
    file_id = '1MFHFz_rxNziYSeN-9ruwnRiYskd0_9ss'
    download_7z(file_id,path)

In [5]:
encoding_4={
    '.':np.array([0],dtype=np.float32),
    'p':np.array([1/12],dtype=np.float32),
    'P':np.array([2/12],dtype=np.float32),
    'b':np.array([3/12],dtype=np.float32),
    'B':np.array([4/12],dtype=np.float32),
    'n':np.array([5/12],dtype=np.float32),
    'N':np.array([6/12],dtype=np.float32),
    'r':np.array([7/12],dtype=np.float32),
    'R':np.array([8/12],dtype=np.float32),
    'q':np.array([9/12],dtype=np.float32),
    'Q':np.array([10/12],dtype=np.float32),
    'k':np.array([11/12],dtype=np.float32),
    'K':np.array([12/12],dtype=np.float32)
}

In [6]:
class Model_4():

    def __init__(self,
                   n_classes=2):
        initializer = tf.keras.initializers.GlorotNormal()
        self.hw=[]
        self.hb=[]

        self.hw.append( tf.Variable(initializer(shape=(7,7,1,32),dtype=np.float32),name="hl1weigths",dtype="float32") )
        self.hb.append( tf.Variable(np.zeros(32,dtype=np.float32),name="hl1bias",dtype="float32") )
        #8x8x32
        self.hw.append( tf.Variable(initializer(shape=(5,5,32,64),dtype=np.float32),name="hl2weigths",dtype="float32"))
        self.hb.append( tf.Variable(np.zeros(64,dtype=np.float32),name="hl2bias",dtype="float32"))
        #8x8x64
        self.hw.append( tf.Variable(initializer(shape=(3,3,64,128),dtype=np.float32),name="hl3weigths",dtype="float32"))
        self.hb.append( tf.Variable(np.zeros(128,dtype=np.float32),name="hl3bias",dtype="float32"))
        #8x8x128
        self.hw.append( tf.Variable(initializer(shape=(8*8*128,256),dtype=np.float32),name="hl4weigths",dtype="float32"))
        self.hb.append( tf.Variable(np.zeros(256,dtype=np.float32),name="hl4bias",dtype="float32"))

        self.hw.append( tf.Variable(initializer(shape=(256, n_classes),dtype=np.float32),name="outweigths",dtype="float32"))
        self.hb.append( tf.Variable(np.zeros(n_classes,dtype=np.float32),name="outbias",dtype="float32"))

        self.trainable_variables = []
        for i in range(len(self.hw)):
            self.trainable_variables.append(self.hw[i])    
            self.trainable_variables.append(self.hb[i])

    def __call__(self,x): 

        out = tf.cast(x, tf.float32)
        out = tf.reshape(out, shape=[-1, 8, 8, 1])

        layer=0
        out = tf.nn.conv2d(out,self.hw[layer], strides=[1,1,1,1], padding='SAME')  
        out = tf.add(out, self.hb[layer])
        out = tf.nn.relu(out)
              #8*8*32
        layer+=1
        out = tf.nn.conv2d(out,self.hw[layer], strides=[1,1,1,1], padding='SAME')  
        out = tf.add(out, self.hb[layer])
        out = tf.nn.relu(out)
              #8*8*64
        layer+=1  
        out = tf.nn.conv2d(out,self.hw[layer], strides=[1,1,1,1], padding='SAME')  
        out = tf.add(out, self.hb[layer])
        out = tf.nn.elu(out)
              #8*8*128
        layer+=1
        out =  tf.reshape(out,[-1, 8*8*128])
        out =  tf.matmul(out,self.hw[layer])
        out = tf.add(out, self.hb[layer])
        out = tf.nn.relu(out)
        out = tf.nn.dropout(out,rate=0.3,seed=0)

        layer+=1
        out =  tf.matmul(out,self.hw[layer])
        out = tf.add(out, self.hb[layer])

        return out

In [7]:
np.random.seed(0)
tf.random.set_seed(0)

x_data = load_pkl(path+name_data)
y_data = load_pkl(path+name_labels)[:,1] #Nota: pasa de onehot a logits

print(x_data.shape)
print(y_data.shape)

x_train, x_test, y_train, y_test = train_test_split(
    x_data, y_data, test_size = test_percent, random_state = 0, shuffle = True)

del x_data
del y_data

(2661290, 64)
(2661290,)


In [8]:

model = Model_4()
encoding=encoding_4


In [9]:
fitting(epochs=epochs,
        x_train=x_train,
        y_train=y_train,
        x_test=x_test,
        y_test=y_test,
        model=model,
        optimizer=optim,
        batch_size=batch_size,
        lr=lr,
        loss_fn=loss_fn,
        save_name=save_name,
        encoding=encoding)

2021-10-30 01:15:47


Epoch: 01/20 | time: 107s = 1.8m | train loss: 0.6247 | train acc: 0.6497 | test loss: 0.6176 | test acc: 0.6558

Epoch: 02/20 | time: 102s = 1.7m | train loss: 0.5942 | train acc: 0.6795 | test loss: 0.5808 | test acc: 0.6920

Epoch: 03/20 | time: 106s = 1.8m | train loss: 0.5614 | train acc: 0.7072 | test loss: 0.5508 | test acc: 0.7155

Epoch: 04/20 | time: 106s = 1.8m | train loss: 0.5177 | train acc: 0.7391 | test loss: 0.5304 | test acc: 0.7309

...

Epoch: 16/20 | time: 100s = 1.7m | train loss: 0.2217 | train acc: 0.9030 | test loss: 0.3132 | test acc: 0.8672

Epoch: 17/20 | time: 100s = 1.7m | train loss: 0.2123 | train acc: 0.9075 | test loss: 0.3205 | test acc: 0.8668

Epoch: 18/20 | time: 99s = 1.7m | train loss: 0.2036 | train acc: 0.9116 | test loss: 0.3012 | test acc: 0.8722

Epoch: 19/20 | time: 99s = 1.7m | train loss: 0.1950 | train acc: 0.9156 | test loss: 0.3128 | test acc: 0.8715

Epoch: 20/20 | time: 99s = 1.7m | train loss: 0.1885 | train acc: 0.9189 | test loss: 0.3000 | test acc: 0.8752

In [10]:
fitting(epochs=20,
        x_train=x_train,
        y_train=y_train,
        x_test=x_test,
        y_test=y_test,
        model= model, 
        load_name=save_name+'.10.h5',
        save_name=save_name,)

2021-10-30 01:49:36


Epoch: 11/30 | time: 100s = 1.7m | train loss: 0.2883 | train acc: 0.8703 | test loss: 0.3558 | test acc: 0.8406

Epoch: 12/30 | time: 99s = 1.7m | train loss: 0.2711 | train acc: 0.8788 | test loss: 0.3522 | test acc: 0.8451

Epoch: 13/30 | time: 99s = 1.7m | train loss: 0.2558 | train acc: 0.8864 | test loss: 0.3383 | test acc: 0.8508

Epoch: 14/30 | time: 99s = 1.7m | train loss: 0.2422 | train acc: 0.8933 | test loss: 0.3272 | test acc: 0.8563

...

Epoch: 26/30 | time: 139s = 2.3m | train loss: 0.1567 | train acc: 0.9336 | test loss: 0.2835 | test acc: 0.8882

Epoch: 27/30 | time: 140s = 2.3m | train loss: 0.1529 | train acc: 0.9356 | test loss: 0.2827 | test acc: 0.8899

Epoch: 28/30 | time: 141s = 2.3m | train loss: 0.1488 | train acc: 0.9373 | test loss: 0.2781 | test acc: 0.8900

Epoch: 29/30 | time: 142s = 2.4m | train loss: 0.1453 | train acc: 0.9389 | test loss: 0.2797 | test acc: 0.8881

Epoch: 30/30 | time: 139s = 2.3m | train loss: 0.1424 | train acc: 0.9402 | test loss: 0.2813 | test acc: 0.8924

In [11]:
fitting(epochs=10,
        x_train=x_train,
        y_train=y_train,
        x_test=x_test,
        y_test=y_test,
        model= model, 
        load_name=save_name+'.30.h5',
        save_name=save_name,)

2021-10-30 02:27:43


Epoch: 31/40 | time: 116s = 1.9m | train loss: 0.1366 | train acc: 0.9426 | test loss: 0.2808 | test acc: 0.8942

Epoch: 32/40 | time: 105s = 1.8m | train loss: 0.1335 | train acc: 0.9441 | test loss: 0.2755 | test acc: 0.8971

Epoch: 33/40 | time: 102s = 1.7m | train loss: 0.1303 | train acc: 0.9456 | test loss: 0.2819 | test acc: 0.8945

Epoch: 34/40 | time: 100s = 1.7m | train loss: 0.1277 | train acc: 0.9467 | test loss: 0.2874 | test acc: 0.8952

...

Epoch: 36/40 | time: 101s = 1.7m | train loss: 0.1225 | train acc: 0.9492 | test loss: 0.2913 | test acc: 0.8965

Epoch: 37/40 | time: 101s = 1.7m | train loss: 0.1206 | train acc: 0.9499 | test loss: 0.2876 | test acc: 0.8954

Epoch: 38/40 | time: 108s = 1.8m | train loss: 0.1194 | train acc: 0.9505 | test loss: 0.2781 | test acc: 0.8981

Epoch: 39/40 | time: 111s = 1.8m | train loss: 0.1170 | train acc: 0.9516 | test loss: 0.2714 | test acc: 0.8992

Epoch: 40/40 | time: 107s = 1.8m | train loss: 0.1151 | train acc: 0.9525 | test loss: 0.2945 | test acc: 0.8961